In [114]:
import sklearn
import pandas as pd
%matplotlib inline

In [115]:
train = pd.read_csv('train.csv')
train.head()
test = pd.read_csv('test.csv')

In [116]:
train['birth_date']
train_len = len(train)

In [117]:
train['brith_date_format'] = pd.to_datetime(train['birth_date'], format='%m/%d/%y')
test['brith_date_format'] = pd.to_datetime(test['birth_date'], format='%m/%d/%y')
date_data = pd.concat([train['brith_date_format'],test['brith_date_format']])

In [118]:
import datetime as DT
import numpy as np
now = pd.Timestamp(DT.datetime.now())
age = date_data.where(date_data < now, date_data -  np.timedelta64(100, 'Y'))   # 2
age = (now - date_data).astype('<m8[Y]')    # 3
age.head()

0    21.0
1    33.0
2    18.0
3    29.0
4    37.0
Name: brith_date_format, dtype: float64

In [119]:
age = pd.Series(age)
age.head()

0    21.0
1    33.0
2    18.0
3    29.0
4    37.0
Name: brith_date_format, dtype: float64

In [120]:
train['age']= age[:train_len]
test['age']= age[train_len:]

In [121]:
train = train.drop(columns=['birth_date','brith_date_format'])
test = test.drop(columns=['birth_date','brith_date_format'])

In [122]:
train.isnull().sum().nlargest(10)
print(train.shape)

(10441, 65)


In [123]:
train_Y = train['y']


In [124]:
corr_mat = train.corr()
means = train.mean()

In [125]:
train = train.fillna(train.mean())
test = test.fillna(test.mean())

In [126]:
train.isnull().sum().nlargest(10)

id             0
club           0
league         0
height_cm      0
weight_kg      0
nationality    0
potential      0
pac            0
sho            0
pas            0
dtype: int64

In [127]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,cross_val_score


In [128]:
train = train.drop(columns=['id','y'])
train.head()
train_x,cv_x,train_y,cv_y = train_test_split(train,train_Y)

In [129]:
train = pd.get_dummies(train)
rf = RandomForestRegressor()

In [130]:

cv = cross_val_score(rf, train, train_Y, scoring='neg_mean_absolute_error') 

In [55]:
print(cv)

[-41.50616202 -38.40826149 -41.73058908]


In [131]:
rf.fit(train,train_Y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [132]:
tid = test['id']
test = test.drop(columns=['id'])
test = pd.get_dummies(test)
test_predict_y = rf.predict(test)

In [138]:
sumbit = pd.DataFrame({'id':tid,'y':test_predict_y})
sumbit.head()

,id,y
0,10441,58.25
1,10442,67.50
2,10443,179.00
3,10444,135.00
4,10445,7.90


In [139]:
sumbit.to_csv('submit.rf.csv')